# Test Data Conventions
What users are able to do: **create / register / get / display / erase / save-load / integrate with product / error cases**.

In [1]:
import sys, os

# compute the parent of tests → FixedIncomeLib
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

print("Added to sys.path:", repo_root)

Added to sys.path: c:\Users\Wanling Xie\FixedIncomeLib


In [2]:
from abc import ABC
import pandas as pd
import json, os
from typing import Dict, Any
from fixedincomelib.apis.data_convention import displayDataConvention
from fixedincomelib.conventions.data_conventions import *

### Create & Display Data Convention
Build one `DataConventionRFR` from a dict and display it.

In [3]:
test_content = {
    "index": "SOFR-1B",
    "accrual_basis": "ACT/360",
    "accrual_period": "3M",
    "payment_offset": "2B",
    "payment_biz_day_conv": "F",
    "payment_hol_conv": "USGS",
    "contractual_notional" : 1000000
}

In [4]:
conv = DataConventionRFRFuture("SOFR-FUTURE-3M", test_content)

In [5]:
displayDataConvention(conv)

,Field,Value
0,unique_name,SOFR-FUTURE-3M
1,data_type,RFR FUTURE
2,index,SOFR-1B
3,accrual_basis,ACT/360
4,accrual_period,3M
5,payment_offset,2B
6,payment_biz_day_conv,F
7,payment_hol_conv,USGS
8,contractual_notional,1000000.0


### DataConventionRegister: Register, List, Insert, Get, Erase

**Register** using DataConvention object function

In [6]:
conv.register_data_conv()

In [7]:
DataConventionRegistry().list_all_convs()

{'USD-SOFR-OIS': <fixedincomelib.conventions.data_conventions.DataConventionRFRSwap at 0x2a247b9dfa0>,
 'SOFR-FUTURE-3M': <fixedincomelib.conventions.data_conventions.DataConventionRFRFuture at 0x2a245dba2d0>}

**Erase** "USD-SOFR-OIS" from the DataConvention Map

In [8]:
DataConventionRegistry().erase('USD-SOFR-OIS')

In [9]:
DataConventionRegistry().list_all_convs()

{'SOFR-FUTURE-3M': <fixedincomelib.conventions.data_conventions.DataConventionRFRFuture at 0x2a245dba2d0>}

**Get (Display)** the details of a data convention

In [10]:
DataConventionRegistry().get("SOFR-FUTURE-3M")

In [ ]:
DataConventionRegistry().get("SOFR-FUTURE-3M")

In [11]:
displayDataConvention(DataConventionRegistry().get("SOFR-FUTURE-3M"))

,Field,Value
0,unique_name,SOFR-FUTURE-3M
1,data_type,RFR FUTURE
2,index,SOFR-1B
3,accrual_basis,ACT/360
4,accrual_period,3M
5,payment_offset,2B
6,payment_biz_day_conv,F
7,payment_hol_conv,USGS
8,contractual_notional,1000000.0


**Insert** a new data convvention

In [12]:
new = {
    "kind": "RFR SWAP",
    "index": "SOFR-1B",
    "accrual_basis": "ACT/360",
    "accrual_period": "1Y",
    "payment_offset": "2B",
    "payment_biz_day_conv": "F",
    "payment_hol_conv": "USGS",
    "ois_compounding": "COMPOUND",
    "contractual_notional" : 1000000
}
new_content = {k:v for k,v in new.items() if k != "kind"}
new_content

{'index': 'SOFR-1B',
 'accrual_basis': 'ACT/360',
 'accrual_period': '1Y',
 'payment_offset': '2B',
 'payment_biz_day_conv': 'F',
 'payment_hol_conv': 'USGS',
 'ois_compounding': 'COMPOUND',
 'contractual_notional': 1000000}

In [13]:
new_conv = DataConventionRFRSwap('XWL-CONV', new_content)
DataConventionRegistry().insert(new_conv)
DataConventionRegistry().list_all_convs

<bound method DataConventionRegistry.list_all_convs of <fixedincomelib.conventions.data_conventions.DataConventionRegistry object at 0x000001E332428800>>

In [14]:
displayDataConvention(DataConventionRegistry().get('XWL-CONV'))

,Field,Value
0,unique_name,XWL-CONV
1,data_type,RFR SWAP
2,index,SOFR-1B
3,accrual_basis,ACT/360
4,accrual_period,1Y
5,payment_offset,2B
6,payment_biz_day_conv,F
7,payment_hol_conv,USGS
8,ois_compounding,COMPOUND
9,contractual_notional,1000000.0
